In [1]:
import copy
import os
import pickle
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset

from utils.options import args_parser
from utils.train_utils import get_data, get_model
from models.Update import DatasetSplit
from models.test import test_img_local, test_img_local_all, test_img_global

import pdb
import easydict

import sys

import random



# Seed
torch.manual_seed(1)#args.running_idx=args.seed
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
np.random.seed(1)
random.seed(1)


In [2]:
def calculate_mean(data):
    return sum(data) / len(data)

def calculate_variance(data):
    mean = calculate_mean(data)
    squared_diff = [(x - mean) ** 2 for x in data]
    variance = sum(squared_diff) / len(data)
    return variance

In [3]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_test, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
                model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 


Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1058], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [4]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_train, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
                model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 


Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1057], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [5]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_test, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
#                 model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 

Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1058], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [6]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_test, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
#                 model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 

Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1058], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [7]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_test, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
#                 model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 

Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1058], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [8]:
model = 'vgg' # cnn, mobile
dataset = 'cifar10' # cifar10, cifar100 
num_classes = 10 # 10, 100
momentum = 0.90
wd = 1e-5
server_data_ratio=0.0


for shard_per_user in [2]:
    for frac in [0.1]:
        for local_ep in [15]:
            for local_upt_part, aggr_part in [('full', 'full')]:
                args = easydict.EasyDict({'epochs': local_ep,
                                          'num_users': 100,
                                          'shard_per_user': shard_per_user,
                                          'server_data_ratio': server_data_ratio,
                                          'frac': frac,
                                          'local_ep': local_ep,
                                          'local_bs': 500,
                                          'bs': 50,
                                          'lr': 0.01,
                                          'momentum': momentum,
                                          'wd': wd,
                                          'model': model,
                                          

                                          'dataset': dataset,
                                          'iid': False,
                                          'num_classes': num_classes,
                                          'gpu': 2,
                                          'verbose': False,
                                          'seed': 1,
                                          'test_freq': 1,
                                          'load_fed': '',
                                          'results_save': 'run1',
                                          'local_upt_part': local_upt_part,
                                          'aggr_part': aggr_part,
                                          'feature_norm': 1,
                                          'fn': False,
                                          'hetero_option': "shard"
                                          })

                # parse args
                args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
                base_dir = './save/full_and_body/{}_iid{}_num{}_C{}_le{}_m{}_wd{}_round_320/shard{}_new/decay_0.1/fn_{}/seed_0/FedAvg'.format(
                    args.model, args.iid, args.num_users, args.frac, args.local_ep, args.momentum, args.wd,args.shard_per_user, args.fn)
                algo_dir = 'local_upt_{}_lr_{}'.format(args.local_upt_part, args.lr)


                dataset_train, dataset_test, dict_users_train, dict_users_test = get_data(args)

                test_dataloader = DataLoader(dataset_train, batch_size=args.bs, shuffle=False)


                # build model
                model = get_model(args)
#                 model_save_path = './full_initial_model.pt'
                model.load_state_dict(torch.load(model_save_path, map_location=args.device), strict=True)

                model.eval()


                # 클래스별로 예측 성공한 데이터와 예측 실패한 데이터의 개수를 저장할 리스트를 생성합니다.
                success_counts = [0 for _ in range(10)]
                failure_counts = [0 for _ in range(10)]

                # 클래스별로 softmax 확률을 누적할 리스트를 생성합니다.
                success_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]
                failure_prob_sums = [torch.zeros(10).to(args.device) for _ in range(10)]

                # 클래스별로 정확한 예측 수를 저장할 리스트를 생성합니다.
                accuracies = [0 for _ in range(10)]


                # 각 데이터의 예측 결과를 확인하고 클래스별로 softmax 확률을 누적합니다.
                with torch.no_grad():
                    for images, labels in test_dataloader:
                        if args.gpu != -1:
                            images, labels = images.to(args.device), labels.to(args.device)

                        outputs = model(images)
                        softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

                        predicted_labels = torch.argmax(softmax_probs, dim=1)

                        for i in range(len(labels)):
                            label = labels[i].item()
                            predicted_label = predicted_labels[i].item()
                            prob_vector = softmax_probs[i]

                            if predicted_label == label:
                                # 예측 성공한 경우
                                success_counts[label] += 1
                                success_prob_sums[label] += prob_vector
                                accuracies[label] += 1
                            else:
                                # 예측 실패한 경우
                                failure_counts[label] += 1
                                failure_prob_sums[label] += prob_vector


            # 클래스별로 평균 softmax 확률 벡터를 계산합니다.
            success_prob_vectors = [success_prob_sums[i] / success_counts[i] for i in range(10)]
            failure_prob_vectors = [failure_prob_sums[i] / failure_counts[i] for i in range(10)]
            classwise_accuracy=[]

            # 결과 출력
            for i in range(10):
                print(f"Class {i+1} - Accuracy: {success_counts[i]/(success_counts[i]+failure_counts[i]):.2%}")
                classwise_accuracy.append(success_counts[i]/(success_counts[i]+failure_counts[i]))
                print("Success Probability Vector:")
                print(success_prob_vectors[i])
                print("Failure Probability Vector:")
                print(failure_prob_vectors[i])
        print(classwise_accuracy)
        print(sum(classwise_accuracy) / len(classwise_accuracy))
        print(calculate_variance(classwise_accuracy))
        print(success_counts) 
        print(failure_counts) 


Files already downloaded and verified
Files already downloaded and verified
Class 1 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0981, 0.0956, 0.0953, 0.0967, 0.1033, 0.1028, 0.1009, 0.1003,
        0.1056], device='cuda:2')
Class 2 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1012, 0.0979, 0.0956, 0.0953, 0.0968, 0.1032, 0.1028, 0.1013, 0.1002,
        0.1057], device='cuda:2')
Class 3 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], device='cuda:2')
Failure Probability Vector:
tensor([0.1014, 0.0982, 0.0958, 0.0954, 0.0967, 0.1033, 0.1028, 0.1007, 0.1004,
        0.1054], device='cuda:2')
Class 4 - Accuracy: 0.00%
Success Probability Vector:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, n